In [1]:
%load_ext rich

# Custom agent


In [2]:
import math

from pydantic import ValidationError

import promptimus as pm

In [3]:
ollama = pm.llms.OpenAILike(
    model_name="gemma3:12b", base_url="http://lilan:11434/v1", api_key="DUMMY"
)

## Tools in the Custom Agent

Tools are modular functions that allow the agent to perform specific calculations or tasks beyond its built-in reasoning capabilities. Each tool is designed to handle a well-defined operation, such as mathematical computations, data transformations, or API interactions. When the agent encounters a query that requires external computation, it selects the appropriate tool, provides structured input, and waits for the tool's response before proceeding.

In [4]:
# you may decorate your function to create a tool
@pm.modules.Tool.decorate
def power(a: float, b: float) -> float:
    """Calcuates the `a` in the pover of `b`"""
    return a**b


def factorial(a: int) -> int:
    """Calcuates the factorial (!) of `a`"""
    return math.factorial(a)


def multiply(a: float, b: float) -> float:
    """Multiplies `a` and `b`"""
    return a * b

In [5]:
# a description for LLM is generated from function docstrings and type hints
print(power.describe())

description = """
## `power` tool.
Calcuates the `a` in the pover of `b`

Parameters:
- `a`: float
- `b`: float
"""




In [6]:
# a tool can be called the same as an original function
power(2, 8)

256.0

In [7]:
# or it can accept a json string in async forward method
await power.forward("""{"a": 2, "b": 8}""")

256.0

In [8]:
# in case of invalid json input the tool will raise pydantic error with clear explanations
try:
    await power.forward("""{"a": 2, "c": 8}""")
except ValidationError as e:
    print(e)

2 validation errors for power
b
  Missing required argument [type=missing_argument, input_value=ArgsKwargs((), {'a': 2, 'c': 8}), input_type=ArgsKwargs]
    For further information visit https://errors.pydantic.dev/2.10/v/missing_argument
c
  Unexpected keyword argument [type=unexpected_keyword_argument, input_value=8, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/unexpected_keyword_argument


## Tool Calling Agent

The Tool Calling Agent is a specialized module that inherits from the core module system, allowing it to manage interactions between an LLM and external tools effectively. It facilitates the selection and execution of tools, ensuring that inputs are validated and requests are properly formatted. In cases where argument parsing errors occur, the agent uses Pydantic error messages to reprompt the model with refined instructions, guiding it to make valid tool calls. This process helps the agent improve its ability to handle complex tool interactions by providing structured feedback.

Additionally, the agent clears extraneous output, such as hallucinations or irrelevant responses, during the parsing step. This ensures the agent’s memory remains clean, valid, and focused on the most relevant information. The agent aggregates the memory module, which stores only the last N messages, maintaining a concise record of recent interactions. 

In [9]:
tool_calling = pm.modules.ToolCallingAgent(
    [power, multiply, factorial], observation_role=pm.MessageRole.USER
).with_llm(ollama)

In [10]:
# all tool descriptions are serialized recursively
print(tool_calling.describe())



[tools.power]
description = """
## `power` tool.
Calcuates the `a` in the pover of `b`

Parameters:
- `a`: float
- `b`: float
"""


[tools.multiply]
description = """
## `multiply` tool.
Multiplies `a` and `b`

Parameters:
- `a`: float
- `b`: float
"""


[tools.factorial]
description = """
## `factorial` tool.
Calcuates the factorial (!) of `a`

Parameters:
- `a`: int
"""



[predictor.prompt]
prompt = """
You are designed to assist with a wide range of tasks—from answering questions and providing summaries to performing detailed analyses—by utilizing a variety of external tools. Follow these strict instructions to ensure correct tool usage and response formatting:

---

## Tools

- **Tool Access:**  
  You have access to multiple tools: 

  {tool_desc}

- **Execution Protocol:**  
  - **One Step at a Time:** In each response, you must either make a single tool call or provide a direct answer to the user.
  - **No Fabrication:** You are strictly forbidden from generating any `Observa

In [11]:
await tool_calling.forward("What is 2 in power of 8?")

Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='256.0', tool_calls=None, tool_call_id=None)

In [12]:
tool_calling.predictor.memory.as_list()


[
    Message(
        role=<MessageRole.USER: 'user'>,
        content='What is 2 in power of 8?',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='Thought:I need to calculate 2 to the power of 8. I will use the power tool for this.\nAction:power\nAction Input: {"a": 2.0, "b": 8.0}',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.USER: 'user'>,
        content='Observation: 256.0',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='Thought:I can answer without using any more tools.\nAnswer:256.0',
        tool_calls=None,
        tool_call_id=None
    )
]

In [13]:
tool_calling.predictor.memory.reset()
await tool_calling.forward("What is a factorial of (2 in power of 3)?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='The factorial of (2 in power of 3) is 40320.',
    tool_calls=None,
    tool_call_id=None
)

In [14]:
tool_calling.predictor.memory.as_list()


[
    Message(
        role=<MessageRole.USER: 'user'>,
        content='What is a factorial of (2 in power of 3)?',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='Thought:I need to first calculate 2 to the power of 3, then calculate the factorial of that result. To do this I will first use the `power` tool and then the `factorial` tool.\nAction:power\nAction Input: {"a": 2, "b": 3}',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.USER: 'user'>,
        content='Observation: 8.0',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='Thought:Now that I have the result of 2 to the power of 3, which is 8, I need to calculate the factorial of 8. I will use the `factorial` tool for this.\nAction:factorial\nAction Input: {"a": 8}',
        tool_calls=None,
        to

In [15]:
tool_calling.predictor.memory.reset()
await tool_calling.forward("What is twice the factorial of 3?")

Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='12.0', tool_calls=None, tool_call_id=None)

In [16]:
tool_calling.predictor.memory.as_list()


[
    Message(
        role=<MessageRole.USER: 'user'>,
        content='What is twice the factorial of 3?',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='Thought:I need to calculate the factorial of 3 first, then multiply the result by 2. I will use the `factorial` tool for the factorial calculation and the `multiply` tool for the multiplication.\nAction:factorial\nAction Input: {"a": 3}',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.USER: 'user'>,
        content='Observation: 6',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='Thought:I have the factorial of 3, which is 6. Now I need to multiply that by 2.\nAction:multiply\nAction Input: {"a": 6.0, "b": 2.0}',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<Messag

## Function calling by OpenAI

To utilze native function calling by OpenAI without ReACT loop use the dedicated `OpenaiToolCallingAgent`

In [17]:
openai = pm.llms.OpenAILike(
    model_name="gpt-4.1-nano",
)

tool_calling = pm.modules.OpenaiToolCallingAgent(
    "Utilize your tools step by step, to act as a calculator.",
    [power, multiply, factorial],
).with_llm(openai)

In [18]:
# all tool descriptions are serialized recursively
print(tool_calling.describe())



[tools.power]
description = """
## `power` tool.
Calcuates the `a` in the pover of `b`

Parameters:
- `a`: float
- `b`: float
"""


[tools.multiply]
description = """
## `multiply` tool.
Multiplies `a` and `b`

Parameters:
- `a`: float
- `b`: float
"""


[tools.factorial]
description = """
## `factorial` tool.
Calcuates the factorial (!) of `a`

Parameters:
- `a`: int
"""



[predictor.prompt]
prompt = """
Utilize your tools step by step, to act as a calculator.
"""

role = """
system
"""




In [19]:
await tool_calling.forward("What is 2 in power of 8?")

/home/ailadin/projects/promptimus/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value <class 'inspect._empty'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)



Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='2 in power of 8 is 256.',
    tool_calls=None,
    tool_call_id=None
)

In [20]:
tool_calling.predictor.memory.as_list()


[
    Message(
        role=<MessageRole.USER: 'user'>,
        content='What is 2 in power of 8?',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='',
        tool_calls=[
            ToolRequest(
                id='call_X1yyNgOPIxcBNaTv3BQcZ8BR',
                type='function',
                function=ToolFunction(name='power', arguments='{"a":2,"b":8}')
            )
        ],
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.TOOL: 'tool'>,
        content='256.0',
        tool_calls=None,
        tool_call_id='call_X1yyNgOPIxcBNaTv3BQcZ8BR'
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='2 in power of 8 is 256.',
        tool_calls=None,
        tool_call_id=None
    )
]

In [21]:
tool_calling.predictor.memory.reset()
await tool_calling.forward("What is a factorial of (2 in power of 3)?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='The value of 2 raised to the power of 3 is 8, and the factorial of 3 is 6.',
    tool_calls=None,
    tool_call_id=None
)

In [22]:
tool_calling.predictor.memory.as_list()


[
    Message(
        role=<MessageRole.USER: 'user'>,
        content='What is a factorial of (2 in power of 3)?',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='',
        tool_calls=[
            ToolRequest(
                id='call_GHFWIYJ4JhJBldGvKrao9SRA',
                type='function',
                function=ToolFunction(name='power', arguments='{"a": 2, "b": 3}')
            ),
            ToolRequest(
                id='call_UxlFyx3hWjLaO6k0IlsLMpGN',
                type='function',
                function=ToolFunction(name='factorial', arguments='{"a": 3}')
            )
        ],
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.TOOL: 'tool'>,
        content='8.0',
        tool_calls=None,
        tool_call_id='call_GHFWIYJ4JhJBldGvKrao9SRA'
    ),
    Message(
        role=<MessageRole.TOOL: 'tool'>,
        content='6',
        tool_calls=None,
    

In [23]:
tool_calling.predictor.memory.reset()
await tool_calling.forward("What is twice the factorial of 3?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Twice the factorial of 3 is 12.',
    tool_calls=None,
    tool_call_id=None
)

In [24]:
tool_calling.predictor.memory.as_list()


[
    Message(
        role=<MessageRole.USER: 'user'>,
        content='What is twice the factorial of 3?',
        tool_calls=None,
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='',
        tool_calls=[
            ToolRequest(
                id='call_v0n3FvrrmTuFrVC7tVp9PXhh',
                type='function',
                function=ToolFunction(name='factorial', arguments='{"a":3}')
            )
        ],
        tool_call_id=None
    ),
    Message(
        role=<MessageRole.TOOL: 'tool'>,
        content='6',
        tool_calls=None,
        tool_call_id='call_v0n3FvrrmTuFrVC7tVp9PXhh'
    ),
    Message(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='',
        tool_calls=[
            ToolRequest(
                id='call_Srj1E4HyC0hxgtWV8ORj7eUL',
                type='function',
                function=ToolFunction(name='multiply', arguments='{"a":2,"b":6}')
            )
        ],
